In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

import dotenv
import loss
import mlflow
import numpy as np
import os
import torch
from torch.utils.data import DataLoader
from intra import IntrA
from functools import partial
from augmentation import apply_augmentations
from models.pointmlp import pointMLP
from models.curvenet import CurveNet
from utils import train_model

# better GPU debugging
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# load environment variables for MLflow tracking
dotenv.load_dotenv()
mlflow.set_tracking_uri(os.environ.get("MLFLOW_TRACKING_URI"))

dev = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Running on {dev}.")

# initialise random seeds
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

torch.backends.cudnn.benchmark = False # select same algo each time code run
torch.backends.cudnn.deterministic = True # make cudnn algos deterministic

In [ ]:
"""Train a CurveNet model on a single fold."""
if True:
    trn = IntrA(
        "/data",
        npoints=1024,
        exclude_seg=True,
        norm=False,
        fold=1,
        kfold_splits="./file_splits/cls",
        test=False,
    )

    tst = IntrA(
        "/data",
        npoints=1024,
        exclude_seg=True,
        norm=False,
        fold=1,
        kfold_splits="./file_splits/cls",
        test=True,
    )
 
    train_dl = DataLoader(trn, batch_size=64, num_workers=8, drop_last=True, shuffle=True)
    test_dl = DataLoader(tst, batch_size=8, num_workers=2, drop_last=False)

    model = pointMLP(num_classes=2)

    train_model(
        model,
        train_dl,
        test_dl,
        loss.smooth_cross_entropy_loss(),
        partial(apply_augmentations),
        model_name="PointMLP",
        epochs=300,
        checkpoint_epoch=150,
        trans_loss=False,
        opt="adamw",
        sched="cosine",
        lr=0.025,
        min_lr=0.0005,
        gamma=0.77,
    )

In [ ]:
"""Train a CurveNet model on a single fold."""
if True:
    trn = IntrA(
        "/data",
        npoints=1024,
        exclude_seg=True,
        norm=False,
        fold=1,
        kfold_splits="./file_splits/cls",
        test=False,
    )

    tst = IntrA(
        "/data",
        npoints=1024,
        exclude_seg=True,
        norm=False,
        fold=1,
        kfold_splits="./file_splits/cls",
        test=True,
    )
 
    train_dl = DataLoader(trn, batch_size=64, num_workers=8, drop_last=True, shuffle=True)
    test_dl = DataLoader(tst, batch_size=8, num_workers=2, drop_last=False)

    model = CurveNet(num_classes=2)

    train_model(
        model,
        train_dl,
        test_dl,
        loss.smooth_cross_entropy_loss(),
        partial(apply_augmentations, rotation=False, translation=False),
        model_name="CurveNet",
        epochs=300,
        checkpoint_epoch=150,
        trans_loss=False,
        opt="sgd",
        sched="cosine",
        lr=0.05,
        min_lr=0.0001,
        gamma=0.67,
    )